In [1]:
import re
import pandas as pd
import seaborn as sns
import time
import matplotlib.pyplot as plt


In [3]:
seqfile = '/global/scratch/projects/fc_mvslab/data/sequencing/CZB_Feb2024/A10_A11/results/A10_1_sequences.txt'
a10design = 'a10_designfile.csv'

In [3]:
def complement(seqn):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N':'N', 'X':'X'} 
    bases = list(seqn) 
    bases = [complement[base] for base in bases] 
    return ''.join(bases)
def reverse_complement(s):
        return complement(s[::-1])

In [8]:
def design_dict(des):
    dt = []
    with open(des, 'r') as f_des:
        # open the design file for reading
        for line in f_des:
            if "ArrayDNA" in line:
                pass
            else:
                dt.append(line.strip())
    
    dt_dict = {}
    for i in dt:
        dt_dict[i] = 1
        
    print(len(dt_dict))
    return(dt_dict)
    
dtiles = design_dict(a10design)

20728


In [7]:
#get list of designed tiles

a10dt = []

with open(a10design, 'r') as f: #read in design file (just arrayDNA column)
    for line in f:
        if "ArrayDNA" in line:
            pass
        else:
            a10dt.append(line.strip())
# print(a10dt[:5])
print(len(a10dt))#20783

#create a dictionary of values
dtiles = {}
for i in a10dt:
    dtiles[i] = 1
    
# print(d)

20728


In [5]:
def tilebc_mapper(read, design_dict,
                  ad_preceder = "GGCTAGC", ad_length = 120, 
                  bc1_preceder = "CGCGCC", bc1length = 11,
                  bc2_preceder = "CTCGAG", bc2length = 14):
    """
    read: 
    ad_preceder:
    ad_length:
    bc1_preceder:
    bc1_length:
    bc2_preceder:
    bc2_length:
    """
    
    #Initial ad search
    searched_read = re.split(ad_preceder, read, maxsplit=1) 
    AD = None
    barcode1 = None
    barcode2 = None
    designed = 0
    
    if len(searched_read) == 2: #if the first ad search was successful
        roi = searched_read[1] #then take the ad search
#         print("roi =", roi)
        AD = roi[:ad_length] #and save the AD as the first part of the string
        
        #Check if AD Tile was designed
        if design_dict:
            if AD in design_dict:
#                 print('yes designed')
                designed = 1
#             else:
#                 print('not designed')

        
        #Initial bc1 search
        searched_read = re.split(bc1_preceder, roi[ad_length:], maxsplit=1) #look for the barcode using the bc preceder in the portion of the read beyond the AD
        if len(searched_read) == 2: #if initial bc1 search was successful
            barcode1 = searched_read[1][:bc1length] #then the barcode is the first 11 characters of the search result
        else: #if bc search via preceder was unsuccessful
            #Secondary bc1 search
            barcode1 = roi[139:150]
    #         print(len(barcode))

        #Initial bc2 search
        searched_read = re.split(bc2_preceder, roi[ad_length:], maxsplit=1) #look for the barcode using the bc preceder in the portion of the read beyond the AD
    #     print(searched_read)
        if len(searched_read) == 2: #if search was successful
            barcode2 = searched_read[1][:bc2length] #bc2 is the first character of search result
        else: #if bc search via preceder was unsuccessful
            #Secondary bc2 search
            barcode2 = roi[168:182]
    
    return AD, barcode1, barcode2, designed

In [6]:
ADeg = reverse_complement('CAtaactaattacatgaGCGGCCNNNNNNNNNNNNNNXTCGAGCTTAAGGGGCCCNNNNNNNNNNNXGCGCGCCTCAGCTAGTCANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNgctagccgatgaagtagagcccgcagt'.upper())
tilebc_mapper(ADeg, dtiles)

('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN',
 'NNNNNNNNNNN',
 'NNNNNNNNNNNNNN',
 0)

In [7]:
def buildmapdf (readfile, readcount):
    """Takes file of sequences only (readfile) and #lines in that file (readcount)"""
    
    #set up progress updates
    iteration_count = 0 #start a counter for the loop to be able to print progress at N iterations
    total_iterations = readcount 
    progress_counter = readcount/10
    
    total_designed = 0 #to immediately count how many tiles were designed
    start1 = time.perf_counter() #start counter
    
    data = []
    
    with open(readfile, 'r') as fin:
        print("Opened file")

        for line in fin:
            """
            Each iteration is 
            """
            iteration_count += 1
            #clean the read
            seq = line.strip()
            seq = reverse_complement(seq)
            #find ad tile and bcs
            to_df = tilebc_mapper(seq, dtiles) #uses gcn4 designed tiles to check for designed
            #count the number of designed
            total_designed += to_df[3]

            if iteration_count % progress_counter ==0: # progress update
                print((f"Iteration {iteration_count}: Progress {iteration_count/total_iterations*100:.2f}%"))
                progtime = time.perf_counter()
                print(f'iteration time progress: {round(progtime-start1, 2)} seconds')

#             print("results = ", ad, adbc, rptrbc, dquery)
            data.append(to_df)
        
    finish = time.perf_counter()
    print(f'finished processing in {round(finish-start1, 2)} seconds')
    print(f'Total designed = {total_designed} out of {total_iterations}')
#     print(data)
    output_df = pd.DataFrame(data, columns=['AD', 'AD_BC', 'RPTR_BC', 'Designed'])
    finish = time.perf_counter()
    print(f'finished creating df in {round(finish-start1, 2)} seconds')
    return output_df

In [8]:
bigmap = buildmapdf(seqfile, 138420998)

Opened file
finished processing in 2540.71 seconds
Total designed = 48675583 out of 138420998
finished creating df in 2614.67 seconds


In [9]:
bigmap.head()

,AD,AD_BC,RPTR_BC,Designed
0,ACTTCTAACTTTCATCATGATCAAGAAGCTGACTTTGTCTTGTTTC...,CCCCGCCATGG,CAGTTATCTCGTTA,0
1,AAAGCTACTGGTCCACCAGCATCTCCAGTTATTAGAGGTTCTGCTT...,TGCATTCCTAC,ACGGATTAGTCGTA,0
2,GGTTCTGGTACTCCACAAAGGAAAGGTTCTGTTGCTTCTACTGGTC...,TTTCAGCTGTT,GAAGGAGCAGGTGC,0
3,AGATTGGAAGCTACTTATGAGAATATTAGAATTAATCAATCTTCTT...,TTCGCTTAATG,AGAAGAACTGAGGA,1
4,TCTTGGTTGCCTTCTTCTTCTTCTTCTCCACCAGCTGCATCTGCTC...,CGTTGCTGACA,GAAGGAAATATGAT,0


In [10]:
bigmap.describe()

,Designed
count,1.384210e+08
mean,3.516488e-01
std,4.774850e-01
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,1.000000e+00
max,1.000000e+00


In [66]:
#plot tile qual as histogram

plt.hist(bigmap['Designed'])

plt.show()

NameError: name 'plt' is not defined

In [11]:
bigmap.to_csv('CURRENT_A10_1_tbcRAW_v2.csv')

In [ ]:
# !pip install scikit-learn
# !pip install ' https://github.com/massivejords/tools/blob/main/dist/labtools-0.1.1-py3-none-any.whl?raw=true'
